In [144]:
import torch
import torch.nn as nn
import numpy as np

# Example of Calculating Softmax
### Softmax can be calculated for Numpy as well as for Tensor

In [145]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [146]:
# for numpy array
x = np.array([2.0, 1.0, 0.1, 1.5, 0.7])
y = softmax(x)
print("softmax y against numpy x: ", y)

softmax y against numpy x:  [0.41727336 0.15350629 0.062411   0.25308909 0.11372026]


In [147]:
# for tensor using torch.softmax method
x = torch.tensor([2.0, 1.0, 0.1, 1.5, 0.7])
y = torch.softmax(x, dim=0)
print("softmax y against tensor x: ", y)

softmax y against tensor x:  tensor([0.4173, 0.1535, 0.0624, 0.2531, 0.1137])


# Crossentropy Loss
### Mostly used for multiclass problem

In [148]:
def crossentropy(y, y_hat):
    loss = -np.sum(y * np.log(y_hat))
    return loss

# One hot encoding for y 

In [149]:
y = np.array([1, 0, 0, 0, 0])

In [150]:
y_pred_good = np.array([0.7, 0.1, 0.05, 0.1, 0.05])
y_pred_bad = np.array([0.1, 0.05, 0.1, 0.05, 0.7])

In [151]:
l1 = crossentropy(y, y_pred_good)
l2 = crossentropy(y, y_pred_bad)
print(f'Numpy Loss 1 (min loss -> good): {l1:.4f}')
print(f'Numpy Loss 2 (max loss) -> bad: {l2:.4f}')

Numpy Loss 1 (min loss -> good): 0.3567
Numpy Loss 2 (max loss) -> bad: 2.3026


# Crossentropy Loss using Pytorch
### Careful!
### nn.CrossEntropyLoss() method applies nn.Softmax() and nn.NLLoss together
### That's why, no need to apply Softmax() at the last layer of the network
### Dont apply One-Hot-Encoding on y before. Just provivde y as a label
### The predicted y should be scored as raw. No Softmax() implementation

In [152]:
loss = nn.CrossEntropyLoss()

### For Single Class

In [153]:
y = torch.tensor([0])

In [154]:
# n_samples x n_classes = 1 x 5
y_pred_good = torch.tensor([[0.7, 0.1, 0.05, 0.1, 0.05]])
y_pred_bad = torch.tensor([[0.1, 0.05, 0.1, 0.05, 0.7]])

In [155]:
l1 = loss(y_pred_good, y)
l2 = loss(y_pred_bad, y)
print("Torch Loss 1 (min loss -> good): ", l1.item())
print("Torch Loss 2 (max loss) -> bad: ", l2.item())

Torch Loss 1 (min loss -> good):  1.1447688341140747
Torch Loss 2 (max loss) -> bad:  1.7447688579559326


In [156]:
_, pred1 = torch.max(y_pred_good, 1)
_, pred2 = torch.max(y_pred_bad, 1)
print(pred1)
print(pred2)

tensor([0])
tensor([4])


### For Multiple Classes

In [157]:
y = torch.tensor([4, 1, 0, 3, 2])

In [158]:
# n_samples x n_classes = 5 x 5
y_pred_good = torch.tensor([[0.05, 0.1, 0.05, 0.1, 0.7], [0.7, 2.1, 0.05, 1.1, 1.3], [4.0, 2.1, 0.05, 1.1, 1.3],
                           [0.1, 1.1, 0.05, 2.3, 1.3], [0.7, 1.2, 4.4, 1.1, 1.3]])
y_pred_bad = torch.tensor([[0.7, 0.1, 0.05, 0.1, 0.05], [2.1, 0.7, 0.05, 1.1, 1.3], [0.05, 2.1, 4.0, 1.1, 1.3],
                           [0.1, 1.1, 0.05, 1.3, 2.3], [0.7, 4.4, 1.2, 1.1, 1.3]])

In [159]:
l1 = loss(y_pred_good, y)
l2 = loss(y_pred_bad, y)
print("Torch Loss 1 (min loss -> good): ", l1.item())
print("Torch Loss 2 (max loss) -> bad: ", l2.item())

Torch Loss 1 (min loss -> good):  0.5913751125335693
Torch Loss 2 (max loss) -> bad:  2.6313750743865967


In [160]:
_, pred1 = torch.max(y_pred_good, 1)
_, pred2 = torch.max(y_pred_bad, 1)
print(pred1)
print(pred2)

tensor([4, 1, 0, 3, 2])
tensor([0, 0, 2, 4, 1])


# Binary Class Classification Using Neural Network
### Sigmoid Function is used instead of Softmax

In [161]:
class NeuralNetBinary(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNetBinary, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, 1)
        
    def forward(self, inputs):
        outputs = self.linear1(inputs)
        outputs = self.relu(outputs)
        outputs = self.linear2(outputs)
        # at the end, sigmoid for binary
        y_hat = torch.sigmoid(outputs)
        return y_hat

In [162]:
model = NeuralNetBinary(input_size=28*28, hidden_size=5)
criterion = nn.BCELoss()

# Multiclass Classification Using Neural Network
### No need to use Softmax, only use Cross Entropy

In [163]:
class NeuralNetMulti(nn.Module):
    def __init__(self, input_size, hidden_size, n_classes):
        super(NeuralNetMulti, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, n_classes)
        
    def forward(self, inputs):
        outputs = self.linear1(inputs)
        outputs = self.relu(outputs)
        outputs = self.linear2(outputs)
        
        return outputs

In [164]:
model = NeuralNetMulti(input_size=28*28, hidden_size=5, n_classes=5)
criterion =  nn.CrossEntropyLoss() # nn.CrossEntropyLoss() automaticsally applies Softmax